## Q5: Implement Kernel PCA for Logistic Regression

Datasets: 

1. Twospirals: 1000 2-dim datapoints

2. Threecircles: 1000 2-dim datapoints



In [2]:
import torchvision
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms


In [ ]:
#A) Train a linear/logistic regression model (library, logistic if data labels are categiories) and confirm that it doesnt work: high classification error or high Root MSE.

class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return torch.sigmoid(self.linear(x))

model = LogisticRegressionModel(28*28)
